# Confirm that noun banks are accessible by ipykernal.
#### - Shared Pandas dfs are transported as pickle files.
#### - Read using the %store -r '...' 'magic method'

In [40]:
# get noun banks
%store -r all_animals
%store -r usa_states
%store -r all_countries
%store -r all_elements
%store -r greek_gods
%store -r roman_gods
%store -r cosmetic_items
%store -r office_supplies

# prevent PyLance from claiming that the variables are not defined (Only necessary for Jupyter Notebooks running in VS Code)
all_animals = all_animals
usa_states = usa_states
all_countries = all_countries
all_elements = all_elements
greek_gods = greek_gods
roman_gods = roman_gods
cosmetic_items = cosmetic_items
office_supplies = office_supplies



# Creating an engine will store a file "nounbanks.db" at SQlite-Databases/nounbanks.db
#### 

In [43]:
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine


def post_to_sqlite():
    # establish database connection
    engine = create_engine('sqlite:///nounbanks.db')
    # populate database with noun banks
    all_animals.to_sql('all_animals', engine, if_exists='replace')
    usa_states.to_sql('usa_states', engine, if_exists='replace')
    all_countries.to_sql('all_countries', engine, if_exists='replace')
    all_elements.to_sql('all_elements', engine, if_exists='replace')
    greek_gods.to_sql('greek_gods', engine, if_exists='replace')
    roman_gods.to_sql('roman_gods', engine, if_exists='replace')
    cosmetic_items.to_sql('cosmetic_items', engine, if_exists='replace')
    office_supplies.to_sql('office_supplies', engine, if_exists='replace')

post_to_sqlite()



In [77]:
import unittest as ut

class TestPostToSqlite(ut.TestCase):
    def __init__(self, *args, **kwargs):
        self.engine = create_engine('sqlite:///nounbanks.db') # create an engine to connect to the database for testing
        super(TestPostToSqlite, self).__init__(*args, **kwargs) # call the parent class's constructor

    def test_connecting_to_sqlite(self):
        self.assertEqual(self.engine.name, 'sqlite') # check that the engine is connected to a sqlite database

    def test_states_start_with_new(self):
        df = pd.read_sql('SELECT state FROM usa_states WHERE state LIKE "New%"', self.engine) # select all states that start with "New"
        self.assertEqual(df.values.tolist(), [['New Hampshire'], ['New Jersey'], ['New Mexico'], ['New York']]) # check that the correct states are returned
        
    def test_longest_len_greek_god(self):
        df = pd.read_sql('SELECT MAX(LENGTH(greek_god)), greek_god FROM greek_gods', self.engine)
        self.assertEqual(df['greek_god'][0], 'hephaestus' ) # check that the greek god with the longest name is Hephaestus
        self.assertEqual(df['MAX(LENGTH(greek_god))'][0], 10) # check that the length of the longest name is 11

    def test_element_with_largest_atomic_number(self):
        df = pd.read_sql('SELECT Element, AtomicNumber FROM all_elements WHERE AtomicNumber = (SELECT MAX(AtomicNumber) FROM all_elements)', self.engine)
        self.assertEqual(df['Element'][0], 'Oganesson') # check that the element with the largest atomic number is Oganesson
        self.assertEqual(df['AtomicNumber'][0], 118) # check that the atomic number of Oganesson is 118


#run the tests

if __name__ == '__main__':
    ut.main(argv=[''], verbosity=3, exit=False)

test_connecting_to_sqlite (__main__.TestPostToSqlite) ... ok
test_element_with_largest_atomic_number (__main__.TestPostToSqlite) ... ok
test_longest_len_greek_god (__main__.TestPostToSqlite) ... ok
test_roman_gods_with_3_letter_names (__main__.TestPostToSqlite) ... FAIL
test_states_start_with_new (__main__.TestPostToSqlite) ... ok

FAIL: test_roman_gods_with_3_letter_names (__main__.TestPostToSqlite)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/nx/n1nd7k552014npzf4p5zpxh00000gn/T/ipykernel_22095/2896697915.py", line 27, in test_roman_gods_with_3_letter_names
    self.assertEqual(df.values.tolist(), [['Aph'], ['Ara'], ['Aur'], ['Bel'], ['Cer'], ['Cup'], ['Dio'], ['Eos'], ['Eur'], ['Fer'], ['Fur'], ['Gan'], ['Hec'], ['Her'], ['Jup'], ['Jut'], ['Lac'], ['Lar'], ['Lib'], ['Lup'], ['Mer'], ['Min'], ['Mun'], ['Nem'], ['Ner'], ['Nix'], ['Oce'], ['Ope'], ['Ori'], ['Pan'], ['Pom'], ['Pri'], ['Pro'], ['Qui'], ['Rap